In [1]:
#!usr/bin/env python  
#coding=utf-8  

from PIL import Image
from threading import Thread
import keyboard
import os
import pyaudio
import subprocess
import time
import wave

In [2]:
########functions for wav playing########
def _wp_init(f):
    print("_wp_init()...")
    time.sleep(2)
    #instantiate PyAudio  
    p = pyaudio.PyAudio()  
    #open stream  
    stream = p.open(format = p.get_format_from_width(f.getsampwidth()),  
                channels = f.getnchannels(),  
                rate = f.getframerate(),  
                output = True)
    return p, stream

def _wp_run(f,c,s):
    print("_wp_run()...")
    data = f.readframes(c)
    while data:
        if keyboard.is_pressed('q'): break
        s.write(data)
        data = f.readframes(c)

def _wp_deInit(s,p,f):
    print("_wp_deInit()...")
    s.stop_stream()  
    s.close()
    f.close()
    p.terminate()

def wavPlaying(file):
    chunk = 1024
    wavfile = wave.open(file,"rb")  
    p, stream = _wp_init(wavfile)
    _wp_run(wavfile,chunk,stream)
    _wp_deInit(stream,p,wavfile)    

########functions for image crop########
def _im_crop_save(src,des,box):
    img = Image.open(src+".png")
    img2 = img.crop(box)
    img2.save(des+".png")

def _im_add_suffix(src,suffix):
    if os.path.isfile(src+'.png'):
        os.rename(src+'.png',src+'_'+suffix+'.png')
    
def _im_cleanup(name):
    for i in range(3):
        for j in ['png','txt']:
            f = name+'_check0'+str(i+1)+'.'+j
            try:
                os.remove(f)
            except OSError as e:
                #print(e)
                pass

def _im_cleanup_extracted_string(s):
    if s[:4] == '_ 一 ' and s[-4:] == ' 一 _': s = s[4:-4]
    if s[:2] == '一 ' and s[-2:] == ' 一': s = s[2:-2]
    if s[:2] == '— ' and s[-2:] == ' —': s = s[2:-2]
    if s[:1] == '_' and s[-1:] == '_': s = s[1:-1]
    return s.strip()
    
def _im_extract_track_artist(src):
    ret, error, track1, track2, artist = True, [], '', '', ''

    _im_crop_save(src, src+"_matchup_track", (120, 1255, 1080, 1365))
    
    track1 = _ocr_to_string(src+"_matchup_track")
    track1 = _im_cleanup_extracted_string(track1)
    if not len(track1): error.append('SCREEN_MATCHUP_EXTRACT_TRACK_NORMAL_FAIL')

    #try with psm7 as single line 
    track2 = _ocr_to_string(src+"_matchup_track",False,'-l chi_sim+eng -psm 7')
    track2 = _im_cleanup_extracted_string(track2)
    if not len(track2): error.append('SCREEN_MATCHUP_EXTRACT_TRACK_PSM7_FAIL')

    _im_crop_save(src, src+"_matchup_artist", (120, 1365, 1080, 1405))
    artist = _ocr_to_string(src+"_matchup_artist")
    artist = _im_cleanup_extracted_string(artist)
    if not len(artist): error.append('SCREEN_MATCHUP_EXTRACT_ARTIST_FAIL')

    if error == []: error.append('SCREEN_MATCHUP_EXTRACT_DONE')
    return ret, error, track1, track2, artist

########functions for image ocr########
def _ocr_to_string(img, cleanup=False, plus='-l chi_sim+eng'):
    subprocess.check_output('tesseract '+img+'.png'+' '+img+' '+plus, shell=True)
    text = ''
    with open(img + '.txt', 'r', encoding='UTF-8') as f:
        text = f.read().strip().replace('\n',' ')
    if cleanup:
        os.remove(img + '.txt')
    return text

########functions for android manipulation########
def _am_formatDuration(d):
    mins = d//60;
    secs = d - mins*60
    return "%02dm%02ds"%(mins,secs)
    
def _am_getDevices():
    cmd = "adb devices"
    out = subprocess.check_output(cmd.split())
    list = out.decode('utf-8').strip().split('\r\n')
    #print(list)
    return list

def _am_init():
    devices = _am_getDevices()
    if len(devices) <= 1 or '\tdevice' not in devices[-1]:
        cmd = "adb kill-server"
        out = subprocess.check_output(cmd.split())
        time.sleep(5)
        cmd = "adb start-server"
        out = subprocess.check_output(cmd.split())
        list = out.decode('utf-8').strip().split('\r\n')
        print(list)
        if 'successfully' not in list[-1]: return False
    return True

def _am_getScreenSize(st):
    print("%s:_am_getScreenSize " % _am_formatDuration(int(time.time()-st)),end='')
    cmd = "adb shell wm size"
    out = subprocess.check_output(cmd.split())
    size = tuple((out.decode('utf-8').strip().split('\r\n')[0].split(' ')[2].split('x')))
    print(size)
    return size

def _am_launchApp(st):
    time.sleep(2)
    print("%s:_am_launchApp" % _am_formatDuration(int(time.time()-st)))

def _am_CaptureFullScreen(des):
    cmd = "adb shell screencap -p /sdcard/screencap.png"
    out = subprocess.check_output(cmd.split())
    cmd = "adb pull /sdcard/screencap.png %s%s" % (des,".png")
    out = subprocess.check_output(cmd.split())
    if 'pulled' not in out.decode('utf-8').strip():
        print("%s:screencap or pull failed, exit..." % _am_formatDuration(int(time.time()-st)))
        return False, "SCREENCAP_OR_PULL_FAILED"
    return True, "SCREENCAP_AND_PULL_DONE"

def _am_pressBackThenRec():
    cmd = "adb shell input tap 40 90"           #this is back arrow position
    out = subprocess.check_output(cmd.split())
    cmd = "adb shell input tap 300 1715"        #this is recognition icon position
    out = subprocess.check_output(cmd.split())

def _am_pressRetry():
    cmd = "adb shell input tap 600 1530"        #this is re-try buttom
    out = subprocess.check_output(cmd.split())
    cmd = "adb shell input tap 300 1715"        #this is recognition icon position
    out = subprocess.check_output(cmd.split())
    
def _am_onScreenAction(fn,ss,st):
    print("%s:_am_onScreenAction " % _am_formatDuration(int(time.time()-st)), end='')
    
    ret, err = True, 'SCREEN_UNKNOWN'
    
    outdir = "_out"
    os.makedirs(outdir, exist_ok=True)

    #caputre the full sceen
    t = _am_formatDuration(int(time.time()-st))
    name = os.path.join(outdir,"%s_%s" % (fn,t))
    ret, err = _am_CaptureFullScreen(name)
    if not ret: return ret,err

    #crop to have multiple check-point sub-images to identify screen
    _im_crop_save(name, name+"_check01", (0, 180, 1200, 240))   #匹配到以下结果
    _im_crop_save(name, name+"_check02", (0, 610, 1200, 675))   #请求超时/未匹配到结果/无网络连接
    _im_crop_save(name, name+"_check03", (0, 1505, 1200, 1555)) #重试/停止识别/开始识别/重新识别/查看识别历史
    #_im_crop_save(name, name+"_check04", (0, 1750, 1200, 1795)) #听歌识曲 哼唱识别

    if '匹配到以下结果' in _ocr_to_string(name+"_check01"):
        #print(" SCREEN_MATCH_UP")
        ret, err, track1, track2, artist = _im_extract_track_artist(name)
        print(err)
        print(" trackX: %s"%track1)
        print(" trackY: %s"%track2)
        print(" artist: %s"%artist)
        _im_add_suffix(name,'matchup')
        time.sleep(5)
        _am_pressBackThenRec()
    elif '未匹配到结果' in _ocr_to_string(name+"_check02"):
        print("SCREEN_NO_MATCH")
        ret, err =  True, 'SCREEN_NO_MATCH'
        _im_add_suffix(name,'nomatch')
        _am_pressBackThenRec()
    elif '请求超时' in _ocr_to_string(name+"_check02"):
        print("SCREEN_REQ_TIMEOUT")
        ret, err =  True, 'SCREEN_REQ_TIMEOUT'
        _im_add_suffix(name,'timeout')
        _am_pressRetry()
    elif '发生错误了' in _ocr_to_string(name+"_check02"):
        print("SCREEN_ERROR_OCCUR")
        ret, err =  True, 'SCREEN_ERROR_OCCUR'
        _im_add_suffix(name,'errOccur')
        _am_pressRetry()
    elif '停止识别' in _ocr_to_string(name+"_check03"):
        print("SCREEN_MATCHING")
        ret, err =  True, 'SCREEN_MATCHING'
        _im_add_suffix(name,'matching')
    else:
        print("SCREEN_UNKNOWN")
        ret, err = True, 'SCREEN_UNKNOWN'
        _im_add_suffix(name,'unkown')
    
    _im_cleanup(name)
    return ret, err

def androidManipulation(t,fn):
    startTime = time.time()
    if not _am_init():
        print("android manipulation exit due to adb init failure...")
        return

    screenSize = _am_getScreenSize(startTime)
    _am_launchApp(startTime)
#    print(t.ident)
    while t.isAlive():
        ret,errorInfo = _am_onScreenAction(fn,screenSize,startTime)
        if not ret:
            print(errorInfo)
            break
        time.sleep(5)

    print("wav playing ends, exit androidManipulation()")

########wav play & android manipulation threads########
def main(f):
    threadWp = Thread(target=wavPlaying, args=(f,))
    threadWp.start()

    threadAm = Thread(target=androidManipulation, args=(threadWp,os.path.splitext(os.path.basename(f))[0],))
    threadAm.start()

    threadWp.join()
    threadAm.join()

In [3]:
wavFolder = 'unrecognized_tracks'
#wavFolder = 'test'

fileList = os.listdir(wavFolder)
wavList = []
for i in fileList:
    ext = os.path.splitext(os.path.basename(i))[1]
    if '.wav' == ext:
        wavList.append(i)

c = 0
for i in sorted(wavList):
    c = c + 1
    print("\n====================")
    print("[%03d: %s]"%(c,i))
    main(os.path.join(wavFolder,i))


[001: cn-t1-01-01_3123_20180718061101.wav_sub_1467_1647.wav]
_wp_init()...
00m00s:_am_getScreenSize ('1200', '1920')
_wp_run()...
00m03s:_am_launchApp
00m03s:_am_onScreenAction SCREEN_MATCHING
00m20s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 阳 光下
 trackY: 阳 光下
 artist: 满江
00m45s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 阳 光下
 trackY: 阳 光下
 artist: 满江
01m09s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 阳 光下
 trackY: 阳 光下
 artist: 满江
01m35s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 阳 光下
 trackY: 阳 光下
 artist: 满江
02m00s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 阳 光下
 trackY: 阳 光下
 artist: 满江
02m25s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 阳 光下
 trackY: 阳 光下
 artist: 满江
02m50s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 阳 光下
 trackY: 阳 光下
 artist: 满江
_wp_deInit()...
wav playing ends, exit androidManipulation()

[002: cn-t1-03-01_3097_20180717071101.wav_sub_1817_1997.wa

00m58s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 想用你名 字命名 的歌
 trackY: 想用你名 字命名 的歌
 artist: 木丽妍
01m23s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 想用你名 字命名 的歌
 trackY: 想用你名 字命名 的歌
 artist: 木丽妍
01m48s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 想用你名 字命名 的歌
 trackY: 想用你名 字命名 的歌
 artist: 木丽妍
02m15s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 想用你名 字命名 的歌
 trackY: 想用你名 字命名 的歌
 artist: 木丽妍
02m41s:_am_onScreenAction _wp_deInit()...
['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 想用你名 字命名 的歌
 trackY: 想用你名 字命名 的歌
 artist: 木丽妍
wav playing ends, exit androidManipulation()

[013: cn-t1-05-01_3091_20180717011103.wav_sub_827_953.wav]
_wp_init()...
00m00s:_am_getScreenSize ('1200', '1920')
_wp_run()...
00m04s:_am_launchApp
00m04s:_am_onScreenAction SCREEN_MATCHING
00m20s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 从此跟着你流浪去很远的地方
 trackY: 从此跟着你流浪去很远的地方
 artist: — ReDor锐豆乐队 一
00m45s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE

01m34s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 你的好
 trackY: 你的好
 artist: 林采欣
02m00s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 你的好
 trackY: 你的好
 artist: 林采欣
02m27s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 你的好
 trackY: 你的好
 artist: 林采欣
02m55s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 你的好
 trackY: 你的好
 artist: 林采欣
03m19s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_ARTIST_FAIL']
 trackX: 我爱你这样深
 trackY: 我爱你这样深
 artist: 
03m44s:_am_onScreenAction SCREEN_MATCHING
04m00s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 快乐星球
 trackY: 快乐星球
 artist: /J、雅
04m27s:_am_onScreenAction SCREEN_MATCHING
04m44s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 快乐星球
 trackY: 快乐星球
 artist: /J、雅
05m09s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 快乐星球
 trackY: 快乐星球
 artist: /J、雅
05m34s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 快乐星球
 trackY: 快乐星球
 artist: /J、雅
06m00s:_am_onScreenAc

00m28s:_am_onScreenAction _wp_deInit()...
['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 贵钞 \卜_ 方 言
 trackY: 贵州 方言
 artist: 三羽 —
wav playing ends, exit androidManipulation()

[029: cn-t1-07-01_3092_20180717021101.wav_sub_1817_2747.wav]
_wp_init()...
00m00s:_am_getScreenSize ('1200', '1920')
_wp_run()...
00m04s:_am_launchApp
00m04s:_am_onScreenAction SCREEN_MATCHING
00m19s:_am_onScreenAction SCREEN_NO_MATCH
00m34s:_am_onScreenAction SCREEN_MATCHING
00m50s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 开往锡城的火车
 trackY: 开往锡城的火车
 artist: 徐艺伦
01m15s:_am_onScreenAction SCREEN_MATCHING
01m32s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 开往锡城的火车
 trackY: 开往锡城的火车
 artist: 徐艺伦
01m57s:_am_onScreenAction SCREEN_MATCHING
02m13s:_am_onScreenAction SCREEN_NO_MATCH
02m28s:_am_onScreenAction SCREEN_MATCHING
02m45s:_am_onScreenAction SCREEN_NO_MATCH
02m59s:_am_onScreenAction SCREEN_MATCHING
03m15s:_am_onScreenAction SCREEN_NO_MATCH
03m29s:_am_onScreenAction SCREEN_MATCHING
03m46s:_

02m55s:_am_onScreenAction _wp_deInit()...
['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: We Are One
 trackY: We Are One
 artist: 那英/ 张惠妹/林忆莲/蔡健雅
wav playing ends, exit androidManipulation()

[036: cn-t1-08-01_3101_20180717111101.wav_sub_1260_1439.wav]
_wp_init()...
00m00s:_am_getScreenSize ('1200', '1920')
_wp_run()...
00m03s:_am_launchApp
00m03s:_am_onScreenAction SCREEN_MATCHING
00m19s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: When _ Was Your Man
 trackY: When _ Was Your Man
 artist: 一 Sam Smith —
00m42s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: When _ Was Your Man
 trackY: When _ Was Your Man
 artist: 一 Sam Smith —
01m07s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: When _ Was Your Man
 trackY: When _ Was Your Man
 artist: 一 Sam Smith —
01m31s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: When _ Was Your Man
 trackY: When _ Was Your Man
 artist: 一 Sam Smith —
01m55s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX:

01m07s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 共圆中国梦
 trackY: 共圆中国梦
 artist: 刘晶/阎维文
01m33s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 共圆中国梦
 trackY: 共圆中国梦
 artist: 刘晶/阎维文
01m58s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 共圆中国梦
 trackY: 共圆中国梦
 artist: 刘晶/阎维文
02m24s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 共圆中国梦
 trackY: 共圆中国梦
 artist: 刘晶/阎维文
02m49s:_am_onScreenAction _wp_deInit()...
['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 共圆中国梦
 trackY: 共圆中国梦
 artist: 刘晶/阎维文
wav playing ends, exit androidManipulation()

[046: cnn-01_3106_20180717154139.wav_sub_2346_2525.wav]
_wp_init()...
00m00s:_am_getScreenSize ('1200', '1920')
_wp_run()...
00m04s:_am_launchApp
00m04s:_am_onScreenAction SCREEN_MATCHING
00m19s:_am_onScreenAction SCREEN_NO_MATCH
00m33s:_am_onScreenAction SCREEN_MATCHING
00m49s:_am_onScreenAction SCREEN_NO_MATCH
01m04s:_am_onScreenAction SCREEN_MATCHING
01m20s:_am_onScreenAction SCREEN_NO_MATCH
01m35s:_am_onScreenA

04m00s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: Shout To The Lord (Shout To The Lord Album Version)
 trackY: Shout TO The Lo「d\多董訾墓言岂菖甄o The Lord Album
 artist: 一 Simp|y Worship Ensemble —
04m27s:_am_onScreenAction SCREEN_MATCHING
04m44s:_am_onScreenAction SCREEN_NO_MATCH
04m59s:_am_onScreenAction SCREEN_MATCHING
05m18s:_am_onScreenAction _wp_deInit()...
SCREEN_NO_MATCH
wav playing ends, exit androidManipulation()

[054: cnn-02_3124_20180718071101.wav_sub_2381_2399.wav]
_wp_init()...
00m00s:_am_getScreenSize _wp_run()...
('1200', '1920')
00m04s:_am_launchApp
00m04s:_am_onScreenAction SCREEN_MATCHING
_wp_deInit()...
wav playing ends, exit androidManipulation()

[055: cnn-02_3124_20180718071101.wav_sub_962_1250.wav]
_wp_init()...
00m00s:_am_getScreenSize ('1200', '1920')
_wp_run()...
00m04s:_am_launchApp
00m04s:_am_onScreenAction SCREEN_NO_MATCH
00m18s:_am_onScreenAction SCREEN_MATCHING
00m37s:_am_onScreenAction SCREEN_NO_MATCH
00m52s:_am_onScreenAction SCREEN_MATCHING

09m54s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: ′_`…肖… 3*" /两康 定
 trackY: 情满康定
 artist: 李志辉
10m21s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: ′_`…肖… 3*" /两康 定
 trackY: 情满康定
 artist: 李志辉
10m48s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: ′_`…肖… 3*" /两康 定
 trackY: 情满康定
 artist: 李志辉
11m14s:_am_onScreenAction SCREEN_MATCHING
11m31s:_am_onScreenAction _wp_deInit()...
['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: ′_`…肖… 3*" /两康 定
 trackY: 情满康定
 artist: 李志辉
wav playing ends, exit androidManipulation()

[061: cnn-07_3094_20180717041101.wav_sub_1494_1674.wav]
_wp_init()...
00m00s:_am_getScreenSize _wp_run()...
('1200', '1920')
00m04s:_am_launchApp
00m04s:_am_onScreenAction SCREEN_MATCHING
00m23s:_am_onScreenAction SCREEN_NO_MATCH
00m38s:_am_onScreenAction SCREEN_MATCHING
00m56s:_am_onScreenAction SCREEN_NO_MATCH
01m12s:_am_onScreenAction SCREEN_MATCHING
01m29s:_am_onScreenAction SCREEN_NO_MATCH
01m45s:_am_onScreenAction SCREEN_MATCHING
02m04s:_am_o

02m39s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 重逢
 trackY: 重逢
 artist: 常石磊/马上又/袁娅维/李佳
03m04s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 重逢
 trackY: 重逢
 artist: 常石磊/马上又/袁娅维/李佳
03m31s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 重逢
 trackY: 重逢
 artist: 常石磊/马上又/袁娅维/李佳
03m56s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 重逢
 trackY: 重逢
 artist: 常石磊/马上又/袁娅维/李佳
04m21s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_DONE']
 trackX: 重逢
 trackY: 重逢
 artist: 常石磊/马上又/袁娅维/李佳
04m47s:_am_onScreenAction SCREEN_MATCHING
05m03s:_am_onScreenAction ['SCREEN_MATCHUP_EXTRACT_TRACK_NORMAL_FAIL', 'SCREEN_MATCHUP_EXTRACT_ARTIST_FAIL']
 trackX: 
 trackY: 信息化尖兵
 artist: 
05m28s:_am_onScreenAction SCREEN_MATCHING
05m44s:_am_onScreenAction SCREEN_NO_MATCH
05m59s:_am_onScreenAction SCREEN_MATCHING
06m14s:_am_onScreenAction SCREEN_NO_MATCH
06m30s:_am_onScreenAction SCREEN_MATCHING
06m46s:_am_onScreenAction SCREEN_NO_MATCH
07m01s:_am_onScreenAction SCREEN